## <span style="color:red">Elementi di programmazione funzionale in Python</span>

In un linguaggio puramente funzionale i programmi sono costituiti solo da un insieme di funzioni (e il "main program" è semplicemente una chiamata di funzione ...) prive di stato interno. Un esempio notevole di linguaggio puramente funzionale è il "glorioso" Lisp, risalente ai primi anni 60 ma che continua ad essere utilizzato sotto forma di dialetti o linguaggi derivati (uno di un certo rilievo è Clojure).

Python chiaramente non è un linguaggio funzionale ma include <u>più di un supporto</u> alla programmazione funzionale. Un elenco non esaustivo, in cui si possono notare argomenti già trattati, include: 

1. Funzioni lambda
2. Decoratori
3. Iteratori e generatori
4. Mapping e riduzione
5. Funzioni di ordine superiore in generale

### Funzioni lambda (richiamo)

In [1]:
docentiInf = ["Paolo Valente", "Manuela Montangero", "Mauro Leoncini", "Giacomo Cabri", "Federica Mandreoli",\
             "Mauro Andreolini", "Riccardo Martoglia", "Marco Villani", "Marko Bertogna", "Nicola Capodieci",\
             "Luca Bedogni", "Andrea Marongiu"]
sorted(docentiInf)

['Andrea Marongiu',
 'Federica Mandreoli',
 'Giacomo Cabri',
 'Luca Bedogni',
 'Manuela Montangero',
 'Marco Villani',
 'Marko Bertogna',
 'Mauro Andreolini',
 'Mauro Leoncini',
 'Nicola Capodieci',
 'Paolo Valente',
 'Riccardo Martoglia']

In [2]:
sorted(docentiInf,key=lambda x: x.split()[-1])  # Sort by family name

['Mauro Andreolini',
 'Luca Bedogni',
 'Marko Bertogna',
 'Giacomo Cabri',
 'Nicola Capodieci',
 'Mauro Leoncini',
 'Federica Mandreoli',
 'Andrea Marongiu',
 'Riccardo Martoglia',
 'Manuela Montangero',
 'Paolo Valente',
 'Marco Villani']

In [14]:
pairs = ((1,4),(-3,5),(2,6),(6,-5))
sorted(pairs, key=lambda x: sum(x))  # Sort by increasing sum

[(6, -5), (-3, 5), (1, 4), (2, 6)]

In [15]:
min(pairs)  # Return the pairs with minimum first component

(-3, 5)

In [16]:
min(pairs,key=lambda x: sum(x))

(6, -5)

### Iteratori e generatori

Iteratori e generatori sono meccanismi che forniscono una funzionalità comune, cioè la capacità di accedere agli elementi di una collezione in sequenza ("uno dopo l'altro uno"( secondo un criterio stabilito.

Sia generatori che iteratori sono costrutti programmativi in grado di preservare uno _stato interno_. Da questo punto di vista, però, gli iteratori consentono di implementare comportamenti più complessi. Un iteratore viene infatti definito a partire da oggetti iterabili, istanze cioè di una classe che implementa il metodo \_\_iter\_\_. In quanto tali, possono definire anche altri comportamenti mediante l'implementazione di altri metodi

Un generatore è realizzato più semplicemente implementando una funzione che, anziché l'istruzione _return_ (esplicita o implicita), utilizza la speciale istruzione _yield_. In questo caso, lo stato è quello della funzione che viene salvato e "ripreso" (resumed) ad ogni invocazione

In [25]:
def Fib_generator():
    x0 = 0
    x1 = 1
    while True:
        yield x0
        x0,x1 = x1,x0+x1

f = Fib_generator()
for i in range(10):
    print(next(f))

0
1
1
2
3
5
8
13
21
34


In [29]:
print(type(Fib_generator))
print(type(f))

<class 'function'>
<class 'generator'>


### Funzioni di ordine superiore (higher-order functions)

Funzioni di ordine superiore sono funzione che hanno funzioni fra i loro parametri e/o restituiscono funzioni come risultato. 

&Egrave; quindi evidente che funzioni di questo tipo sono già state affrontate esaustivamente in questo corso (trattando di decoratori, funzioni lambda iteratori e generatori). 

### Lo schema algoritmico map-reduce

Si tratta di un procedimento di calcolo tipico dei linguaggi funzionali e presente in Python (con qualche nervosismo di Guido Van Rossum...). 

Un primo semplice esempio, che comunque illustra la "struttura" del paradigma, riguarda il <em>calcolo della somma dei quadrati dei primi n numeri naturali</em>

In [30]:
# Calcolo della somma dei primi n numeri naturali: versione "tipo c"
def sum_of_squares(n):
    s = 0                     # Accumulatore
    for i in range(1,n+1):
        s += i*i
    return s

In [35]:
sum_of_squares(10)

385

In [36]:
# Calcolo della somma dei primi n numeri naturali: versione 1.0 map-reduce
def sum_of_squares_MR(n):
    s = [i*i for i in range(1,n+1)] # map step
    return sum(s)                   # reduce step

In [38]:
sum_of_squares_MR(10)

385

La seconda versione illustra i due step dello schema map-reduce
1. passo <em>map</em>; una stessa operazione viene applicata singolarmente ad ogni dato che compone la collezione di input
2. passo <em>reduce</em>; un'operazione (tipicamente binaria) viene applicata ai dati prodotti in output da map in modo da produrre un unico risultato in uscita

La versione MR presenta il chiaro svantaggio di richiedere memoria $O(n)$, al contrario della prima che usa una quantit costante di spazio. Tuttavia, la versione MR, in particolare proprio il primo passo, si presta molto bene ad essere parallelizzato. Implementazioni di questo schema si trovano non a caso in sistemi generali di calcolo distribuito come Spark e Hadoop di Apache. In contesti reali, lo schema può essere più complesso è precedere anche un passo intermedio di _shuffling_ Vedremo il ruolo di quest'ultimo trattando un secondo esempio poco più avanti.

#### Supporto "di base" per map-reduce in Python

In [63]:
# Calcolo della somma dei primi n numeri naturali: versione 1.0 map-reduce
def sum_of_squares_MR(n):
    from functools import reduce
    s=map(lambda i: i*i,range(1,n+1))            # map step
    return reduce(lambda x,y:x+y,s)              # reduce step

In [66]:
sum_of_squares_MR(10)

385

In questa seconda versione MR, il passo map viene eseguito utilizzando la omonima funzione (di ordine superiore).

1. map riceve in input una funzione e un iterabile e applica la funzione a tutti gli elementi dell'iterabile restituendo, in output, un secondo iterabile con i valori calcolati
2. ricevuta in input una funzione binaria $f$ e un iterabile $I$, reduce esegue invece i seguenti $|I|-1$ passi:
  
  2.1 al primo passo applica $f$ ai primi due elementi di $I$
  
  2.2 al passo $k$, $k=2,\ldots,|I|-1$, applica $f$ al risultato ottenuto al passo $k-1$ e al $k+1$-esimo elemento di $I$
  
Il seguente schema illustra il funzionamento di reduce su una sequenza di 4 elementi

<img src="reduce.jpg">

In [13]:
# Un altro esempio di uso di reduce: il calcolo del fattoriale
from functools import reduce
reduce(lambda x,y: x*y,range(1,10)) 

362880

#### Un esempio un po' più complesso (considerato comunque, in relazione a map-reduce; l'equivalente di "Hello World!" per chi inizia a programmare in un nuovo linguaggio): il calcolo delle occorrenze delle varie parole in in testo

In questo caso, i dati da manipolare sono coppie (chiave,valore) (le chiavi sono le parole, i valori i corrispondenti conteggi). Iniziamo anche qui con una soluzione "c-like", senza cioè utilizzare lo schema map-reduce

In [73]:
# Definiamo dapprima una funzione per recuperare le parole da una stringa
import re
def getwords(S):
    '''Data una stringa S, dapprima elimina tutto ciò che non è word all'inizio e alla fine di S,
       dopodiché restituisce un iterabile con le word (in carattere minuscolo) di S
    '''
    S = re.split(r'\W+',re.sub(r'^\W+|\W+$', '', S))
    return map(str.lower, S)

In [74]:
for w in getwords('\t . pippo pluto e pure \n\t topolino   \t'):
    print(w)

pippo
pluto
e
pure
topolino


In [78]:
# Soluzione Python C-like
with open('Alice.txt', 'r') as input_file:    # Alice's Adventures in Wonderland by L. Carroll
    words = {}
    for line in input_file:
        if line != '\n':
            for w in getwords(line):
                words[w] = words.get(w, 0) + 1

In [76]:
#Possiamo adesso, ad esempio, controllare quali sono le 10 parole più frequenti nel testo
mostfreqwords = sorted(words.keys(), key=lambda w: words[w], reverse=True)[:10]

for w in mostfreqwords:
    print(f"{w} : \t appers {words[w]} times")

Vogliamo ora applicare lo schema algoritmico map-reduce, transitando però attraverso una versione che è "memory inefficient".

1. Come passo iniziale, dobbiamo trasformare ogni occorrenza di una parola _w_ nella coppia chiave,valore  (_w_,1), dove 1 indica esattamente una occorrenza. Questa è la **mappatura**

In [193]:
words = []
with open('Alice.txt', 'r') as input_file: 
    for line in input_file:
        if line != '\n':
            words += getwords(line)
mappedseq = map((lambda w : (w, 1)), words)

2. Il secondo passo è il già anticipato **shuffling**. Il rimescolamento in questo caso deve portare tutte le coppie con la stessa chiave ad essere fra loro vicine. In questo caso non è necessario specificare una funzione lambda che definisca la chiave da usare; infatti, per default, l'ordinamento di tuple avviene sulla base della prima componente (in questo caso proprio la chiave).

In [194]:
sortedmappedseq= sorted(mappedseq)

3. Il terzo passo naturalmente è la **riduzione**. In questo caso la semplice reduce non serve al nostro scopo perchè essa produrrebbe una singola coppia. Abbiamo invece bisogno di effettuare una riduzione che "raggruppi" tutte le coppie con la stessa chiave (sommando le frequenza). Quel che ci serve è, appunto, la funzione (reminiscente di SQL) _groupby__

In [86]:
from itertools import groupby

In [87]:
help(groupby)

Help on class groupby in module itertools:

class groupby(builtins.object)
 |  groupby(iterable, key=None) -> make an iterator that returns consecutive
 |  keys and groups from the iterable.  If the key function is not specified or
 |  is None, the element itself is used for grouping.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



Prima un esempio con pochi elementi, meglio visualizzabili

In [217]:
G = [('a',1),('a',1),('b',1),('c',1),('c',1),('c',1)]

In [218]:
raggruppamento = groupby(G,key=lambda x: x[0])

In [181]:
list(raggruppamento)

[('a', <itertools._grouper at 0x7efc74b77150>),
 ('b', <itertools._grouper at 0x7efc74b77ed0>),
 ('c', <itertools._grouper at 0x7efc74b77e50>)]

In [182]:
raggruppamento = groupby(G,key=lambda x: x[0])
for t in map(lambda l: (l[0], reduce(lambda x, y: x + y, map(lambda p: p[1], l[1]))), raggruppamento):
    print(t)

('a', 2)
('b', 1)
('c', 3)


Ritorniamo ora al nostro codice

In [213]:
groups = groupby(sortedmappedseq,key=lambda x: x[0])

In [214]:
wordcounts = map(lambda l: (l[0], reduce(lambda x, y: x + y, map(lambda p: p[1], l[1]))), groups)

In [215]:
wordcounts

In [210]:
for i in range(10):
    print(next(wordcounts))

('', 9)
('_i_', 2)
('a', 632)
('abide', 1)
('able', 1)
('about', 94)
('above', 3)
('absence', 1)
('absurd', 2)
('acceptance', 1)


In [216]:
sorted(wordcounts,key=lambda x: x[1],reverse=True)[:10]

[('the', 1642),
 ('and', 872),
 ('to', 729),
 ('a', 632),
 ('it', 595),
 ('she', 553),
 ('i', 543),
 ('of', 514),
 ('said', 462),
 ('you', 411)]

Versione efficiente in spazio e "distribuibile"

In [219]:
# Mappatura su file
import re
words = {}
with open('Alice.txt','r') as input_file:
    with open('Alice.map','w') as map_file:
        for line in input_file:
            for w in getwords(line):
                map_file.write(w.lower()+"\t1\n") # I due campi (word,count) sono separati da un tab

In [ ]:
# Ordinamento (shuffling) usando memoria centrale "controllata"
# See https://stackoverflow.com/questions/14465154/sorting-text-file-by-using-python/16954837#16954837

import sys
from functools import partial
from heapq import merge
from tempfile import TemporaryFile

# defininiamo il criterio di sorting
def sortkey(line):
    return line.split('\t')[0]

with open('Alice.map') as infile:
    # sort lines in small batches, write intermediate results to temporary files
    sorted_files = []
    nbytes = 1 << 16 # Carichiamo 64KB per volta 
    for lines in iter(partial(infile.readlines, nbytes), []):
        lines.sort(key=sortkey)     # ordiniamo in memoria il batch (di al più nbytes) attuale
        f = TemporaryFile("w+")
        f.writelines(lines)
        f.seek(0) # rewind
        sorted_files.append(f)

# fusione e scrittura dei risultati
with open('Alice.map.sorted','w') as sortedmap:
    sortedmap.writelines(merge(*sorted_files, key=sortkey))

# clean up
for f in sorted_files:
    f.close() # chiudiamo e cancelliamo i file temporanei

In [ ]:
# Fase di riduzione. Semplice perché il file è ordinato
# Lasciata per esercizio